In [19]:
import pandas as pd

In [20]:
PATH = "gs://induction-labs/evals/checkpoints/merged_step_-1_uUhioFvR_step_868_Nik8u94a/2025-08-04T04-31-39/osworld_eval_8yv5xv_j"
DATA_FILE = f"{PATH}/samples.jsonl"

In [3]:
# fix samples
ORIGINAL_TRAJECTORIES = pd.read_json(DATA_FILE, lines=True)
FIXED_TRAJECTORIES = ORIGINAL_TRAJECTORIES.copy()
FIXED_TRAJECTORIES["eval_task_id"] = FIXED_TRAJECTORIES["eval_task_id"].str.replace(r"-2$", "", regex=True)
FIXED_TRAJECTORIES = FIXED_TRAJECTORIES[(FIXED_TRAJECTORIES["reward"] == 0) | (FIXED_TRAJECTORIES["reward"] == 1)]

In [4]:
from google.cloud import storage
import asyncio
import json
import re

_GS_RE = re.compile(r"^gs://([^/]+)/(.+)$")
def load_turns_gcs(gs_uri: str):
    """
    Reads a JSON array from Google Cloud Storage and returns
    [{"image": ..., "text": ...}, ...] minus the last row.
    """
    m = _GS_RE.match(gs_uri)
    if not m:
        raise ValueError(f"Not a valid gs:// URI: {gs_uri}")
    bucket_name, blob_name = m.groups()

    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # download the whole object as one string
    data_str = blob.download_as_text()

    records = json.loads(data_str)  # list-of-dicts
    return [
        {"image": r["image"], "text": r["text"]} for r in records[:-1]
    ]  # mimic .iloc[0:-1]

async def load_turns_gcs_async(semaphore: asyncio.Semaphore, gs_uri: str):
    """
    Asynchronous version of load_turns_gcs.
    """
    async with semaphore:
        return await asyncio.to_thread(load_turns_gcs, gs_uri)

In [5]:
legacy = False
if legacy:
    semaphore = asyncio.Semaphore(64)  # limit concurrent loads
    async def check_if_fail(semaphore, attempt_id):
        try:
            return (await load_turns_gcs_async(semaphore, f"{PATH}/metadata/{attempt_id}.json"))[-1]["text"] is None
        except:
            return True

    failed = await asyncio.gather(*[
        check_if_fail(semaphore, row["attempt_id"])
        for _i, row in FIXED_TRAJECTORIES.iterrows()
    ])

In [6]:
if legacy:
    FIXED_TRAJECTORIES["failed"] = failed
    FIXED_TRAJECTORIES = FIXED_TRAJECTORIES[~FIXED_TRAJECTORIES["failed"]]

In [7]:
FIXED_TRAJECTORIES.to_json(f"{PATH}/samples_fixed.jsonl", orient="records", lines=True)

In [21]:
len(FIXED_TRAJECTORIES)

749

In [18]:
len(ORIGINAL_TRAJECTORIES)

1290

In [3]:
# FIXED_TRAJECTORIES = pd.read_json(f"{PATH}/samples_fixed.jsonl", lines=True)
import pandas as pd
PATH = "gs://induction-labs/passive_data/2025-08-13/all_data_reprocessed_good-04-19-14".rstrip("/")
# PREFIX = "hard"
FIXED_TRAJECTORIES = pd.read_json(f"{PATH}/samples.jsonl", lines=True)
FIXED_TRAJECTORIES['NUM_HOTKEYS'] = FIXED_TRAJECTORIES['actions'].apply(lambda x: sum(k.count('hotkey(key=') for k in x))
print(FIXED_TRAJECTORIES['NUM_HOTKEYS'].describe())
FIXED_TRAJECTORIES = FIXED_TRAJECTORIES[FIXED_TRAJECTORIES['NUM_HOTKEYS'] <= 5]
# FIXED_TRAJECTORIES['actions'].iloc[0][0]
# FIXED_TRAJECTORIES = FIXED_TRAJECTORIES.drop(columns=["thinking", "actions"])

count    392.000000
mean       1.459184
std        1.380673
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        4.000000
Name: NUM_HOTKEYS, dtype: float64


In [22]:
FIXED_TRAJECTORIES = FIXED_TRAJECTORIES[FIXED_TRAJECTORIES['NUM_HOTKEYS'] <= 5]
FIXED_TRAJECTORIES['NUM_HOTKEYS'].describe()

count    392.000000
mean       1.459184
std        1.380673
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        4.000000
Name: NUM_HOTKEYS, dtype: float64

In [4]:
FIXED_TRAJECTORIES['trajectory_length'] = FIXED_TRAJECTORIES['trajectory_length'] - 1

In [24]:
FIXED_TRAJECTORIES

,attempt_id,eval_task_id,actions,thinking,instruction,trajectory_length,source_dir,image_turns_start,image_turns_end,text_turns_start,text_turns_end,unmask_last_only,NUM_HOTKEYS
0,gCkzE6WKvmCx,gCkzE6WKvmCx,"[left_double(start_box='(100,732)'), left_doub...","[I need to swap out “Aman” for “Ankit,” so I s...",Update the draft email by replacing “Aman” wit...,10,gs://induction-labs-data-ext/action_capture/ar...,0,10,0,10,False,0
1,lwZzEnH66DGd,lwZzEnH66DGd,"[drag(start_box='(242,897)', end_box='(0,897)'...",[To replace “Is this relevant?” with the new p...,"Open Kevin Long’s email task in Amplemarket, h...",10,gs://induction-labs-data-ext/action_capture/ar...,0,10,0,10,False,1
2,B0kJBrWVdnMa,B0kJBrWVdnMa,"[type(content='<Backspace>'), type(content=' o...",[To attach “ once they are!” immediately after...,Edit the draft email by replacing the current ...,10,gs://induction-labs-data-ext/action_capture/ar...,0,10,0,10,False,2
3,TNb1ZZTdfmj1,TNb1ZZTdfmj1,"[click(start_box='(242,1067)'), click(start_bo...",[This draft already has the integrations subje...,Send the “integrations” follow-up to today’s l...,10,gs://induction-labs-data-ext/action_capture/ar...,0,10,0,10,False,2
4,0rMc31Z1HW4B,0rMc31Z1HW4B,"[type(content='<Backspace>'), hotkey(key='v', ...",[I have to clear out this old email before I i...,"Clear the draft, paste the new Brain MAX integ...",10,gs://induction-labs-data-ext/action_capture/ar...,0,10,0,10,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,a8zwnTGSe9Pr,a8zwnTGSe9Pr,"[scroll(direction='up', start_box='(592,675)')...",[I need to reach the point where the evaluatio...,"In page.tsx, scroll to the start of the evalua...",10,gs://induction-labs-data-ext/action_capture/jo...,0,10,0,10,False,1
388,RIsDanpzI1NE,RIsDanpzI1NE,"[click(start_box='(1149,70)'), hotkey(key='f',...",[I need to search for the UUID in the Sample T...,"Open the “Sample Tasks” Google Sheet, search (...",10,gs://induction-labs-data-ext/action_capture/jo...,0,10,0,10,False,2
389,AbD5740UhfTY,AbD5740UhfTY,"[drag(start_box='(1011,126)', end_box='(777,12...",[The Chrome New Tab space is sitting near the ...,Switch to Google Chrome through Mission Contro...,10,gs://induction-labs-data-ext/action_capture/Ja...,0,10,0,10,False,2
390,Fqccj4hMblVZ,Fqccj4hMblVZ,"[click(start_box='(740,489)'), click(start_box...",[The new “if” stub must sit right after setIsE...,"In the AnnotationInterface component, place th...",10,gs://induction-labs-data-ext/action_capture/jo...,0,10,0,10,False,0


In [8]:
import pandas as pd
import numpy as np

def expand_with_sliding_windows(df: pd.DataFrame, width: int,
                                length_col: str = "trajectory_length") -> pd.DataFrame:
    """
    For every row in *df*, create (trajectory_length – width + 1) copies,
    one for every sliding window of size *width*.
    Adds two new columns:  `window_start` and `window_end` (inclusive).
    Rows whose `trajectory_length` < width are silently dropped.

    Parameters
    ----------
    df : pd.DataFrame                 Original data.
    width : int                       Size of the sliding window.
    length_col : str, default 'trajectory_length'
                                      Column holding the total length.

    Returns
    -------
    pd.DataFrame                      Expanded frame.
    """
    if width <= 0:
        raise ValueError("width must be a positive integer")

    # how many windows each row will produce
    n_windows = df[length_col] - width + 1

    # keep only rows where at least one window fits
    valid_mask = n_windows > 0
    df_valid   = df[valid_mask].copy()
    n_windows  = n_windows[valid_mask]

    # repeat the rows the right number of times
    expanded = df_valid.loc[df_valid.index.repeat(n_windows)].reset_index(drop=True)
    rng = np.random.default_rng(42)

    # build the window start indices in one NumPy shot
    starts = np.concatenate([np.arange(k) for k in n_windows])



    expanded["image_turns_end"]   = starts + width        # inclusive upper bound
    expanded["text_turns_end"]   = starts + width
    expanded["rand_turns_start"] = rng.integers(0, (expanded["image_turns_end"] -1).to_numpy(), endpoint=True)
    expanded["text_turns_start"] = np.maximum(0, expanded["rand_turns_start"])
    expanded["image_turns_start"] = np.maximum(starts, expanded["rand_turns_start"])

    expanded["unmask_last_only"] = None
    expanded["unmask_last_n_turns"] = -1
    expanded["unmask_last_n_turns"][expanded["image_turns_end"] > 5] = 1

    return expanded

In [9]:
CORRECT_TRAJECTORIES_EXPANDED = FIXED_TRAJECTORIES.copy()
CORRECT_TRAJECTORIES_EXPANDED = CORRECT_TRAJECTORIES_EXPANDED #[CORRECT_TRAJECTORIES_EXPANDED["reward"] == 1]
CORRECT_TRAJECTORIES_EXPANDED = expand_with_sliding_windows(
    CORRECT_TRAJECTORIES_EXPANDED,
    width=5,
    length_col="trajectory_length"
)
# num = 50
# CORRECT_TRAJECTORIES_EXPANDED = CORRECT_TRAJECTORIES_EXPANDED[CORRECT_TRAJECTORIES_EXPANDED["text_turns_end"] <= num]
CORRECT_TRAJECTORIES_EXPANDED

/tmp/ipykernel_2972234/1651631468.py:51: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  expanded["unmask_last_n_turns"][expanded["image_turns_end"] > 5] = 1
/tmp/ipykernel_2972234/1651631468.py:51: SettingWithCopyWarning: 
A value is trying t

,attempt_id,eval_task_id,actions,thinking,instruction,trajectory_length,source_dir,image_turns_start,image_turns_end,text_turns_start,text_turns_end,unmask_last_only,NUM_HOTKEYS,rand_turns_start,unmask_last_n_turns
0,gCkzE6WKvmCx,gCkzE6WKvmCx,"[left_double(start_box='(100,732)'), left_doub...","[I need to swap out “Aman” for “Ankit,” so I s...",Update the draft email by replacing “Aman” wit...,10,gs://induction-labs-data-ext/action_capture/ar...,0,5,0,5,None,0,0,-1
1,gCkzE6WKvmCx,gCkzE6WKvmCx,"[left_double(start_box='(100,732)'), left_doub...","[I need to swap out “Aman” for “Ankit,” so I s...",Update the draft email by replacing “Aman” wit...,10,gs://induction-labs-data-ext/action_capture/ar...,4,6,4,6,None,0,4,1
2,gCkzE6WKvmCx,gCkzE6WKvmCx,"[left_double(start_box='(100,732)'), left_doub...","[I need to swap out “Aman” for “Ankit,” so I s...",Update the draft email by replacing “Aman” wit...,10,gs://induction-labs-data-ext/action_capture/ar...,4,7,4,7,None,0,4,1
3,gCkzE6WKvmCx,gCkzE6WKvmCx,"[left_double(start_box='(100,732)'), left_doub...","[I need to swap out “Aman” for “Ankit,” so I s...",Update the draft email by replacing “Aman” wit...,10,gs://induction-labs-data-ext/action_capture/ar...,3,8,3,8,None,0,3,1
4,gCkzE6WKvmCx,gCkzE6WKvmCx,"[left_double(start_box='(100,732)'), left_doub...","[I need to swap out “Aman” for “Ankit,” so I s...",Update the draft email by replacing “Aman” wit...,10,gs://induction-labs-data-ext/action_capture/ar...,4,9,3,9,None,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,JzMKJBLF7FQm,JzMKJBLF7FQm,"[hotkey(key='v', modifiers='cmd'), type(conten...",[I already copied the draft of the new async o...,Replace the green “Check” button’s `onClick` h...,10,gs://induction-labs-data-ext/action_capture/jo...,4,6,4,6,None,1,4,1
2348,JzMKJBLF7FQm,JzMKJBLF7FQm,"[hotkey(key='v', modifiers='cmd'), type(conten...",[I already copied the draft of the new async o...,Replace the green “Check” button’s `onClick` h...,10,gs://induction-labs-data-ext/action_capture/jo...,2,7,1,7,None,1,1,1
2349,JzMKJBLF7FQm,JzMKJBLF7FQm,"[hotkey(key='v', modifiers='cmd'), type(conten...",[I already copied the draft of the new async o...,Replace the green “Check” button’s `onClick` h...,10,gs://induction-labs-data-ext/action_capture/jo...,4,8,4,8,None,1,4,1
2350,JzMKJBLF7FQm,JzMKJBLF7FQm,"[hotkey(key='v', modifiers='cmd'), type(conten...",[I already copied the draft of the new async o...,Replace the green “Check” button’s `onClick` h...,10,gs://induction-labs-data-ext/action_capture/jo...,4,9,4,9,None,1,4,1


In [10]:
save_path = f"{PATH}/random_expanded_samples.jsonl"
CORRECT_TRAJECTORIES_EXPANDED.to_json(f"{save_path}", orient="records", lines=True)
print(f"Saved {len(CORRECT_TRAJECTORIES_EXPANDED)} expanded trajectories to {save_path}")

Saved 2352 expanded trajectories to gs://induction-labs/passive_data/2025-08-13/all_data_reprocessed_good-04-19-14/random_expanded_samples.jsonl


In [ ]:
shuffled = CORRECT_TRAJECTORIES_EXPANDED.sample(frac=1, random_state=248239)
test_size = 32
train_size = len(shuffled) - test_size
TRAIN_SET = shuffled.iloc[:train_size]
TEST_SET = shuffled.iloc[train_size:]
TEST_SET = pd.concat([TEST_SET], ignore_index=True)

TRAIN_SET.to_json(f"{PATH}/{PREFIX}_samples_correct_trajectories_expanded_under_{num}_train.jsonl", orient="records", lines=True)
TEST_SET.to_json(f"{PATH}/{PREFIX}_samples_correct_trajectories_expanded_under_{num}_test.jsonl", orient="records", lines=True)

In [ ]:
# pd.concat([TRAIN_SET] * 10, ignore_index=True).to_json(f"{PATH}/{PREFIX}_samples_correct_trajectories_expanded_under_{num}_train_10x.jsonl", orient="records", lines=True)

In [14]:
len(TRAIN_SET)/32

46.0